# Detección de color rojo en tiempo real y cálculo del espacio de color en RGB

**Curso:** Detección de colores en tiempo real utilizando Python y OpenCV </br>
**Autor:** Jesús Guadalupe Pérez Flores</br>
**Lugar:** Mineral de la Reforma, Hidalgo, México. </br>
**Fecha:** Octubre 2023 </br>

## Descripción
Este código utiliza OpenCV (Open Source Computer Vision Library) y Python para detectar el color rojo en tiempo real desde una cámara web y después calcula su espacio de color y lo reprtra en el formato RGB. 
A continuación, te explico las partes clave del código:

### 1. Importación de bibliotecas
Se importan las bibliotecas necesarias:
- `cv2`: Esta es la biblioteca OpenCV que se utiliza para el procesamiento de imágenes y video.
- `numpy`: Se utiliza para operaciones matemáticas y manipulación de matrices.

In [1]:
import cv2
import numpy as np

### 2. Definir una función para detectar objetos de color rojo y calcular el espacio de color promedio en RGB
- Esta función toma un frame de imagen como entrada y realiza la detección de objetos de color rojo en ese frame.
- El frame se convierte primero del espacio de color BGR al espacio de color HSV (Hue, Saturation, Value), lo que facilita la detección de colores.
- Se definen dos rangos de valores HSV para el color rojo, ya que el color rojo se encuentra en dos extremos del espectro de colores. Estos rangos se almacenan en `lower_red` y `upper_red`.
- Se crean dos máscaras `mask1` y `mask2` utilizando estos rangos para aislar las áreas de color rojo en el frame.
- Luego, las dos máscaras se combinan en una sola máscara llamada mask.
- Se aplican operaciones morfológicas para eliminar el ruido en la máscara.
- Se encuentran los contornos en la máscara que representan objetos de color rojo.
- Para cada contorno, si su área es mayor que 1000 píxeles, se dibuja un rectángulo alrededor del objeto rojo en el frame y se calcula el espacio de color promedio dentro de ese rectángulo en el espacio de color BGR (Blue, Green, Red). El espacio de color promedio se muestra como una etiqueta en el frame.

In [ ]:
def detect_red_objects(frame):
    # Convertir el frame a espacio de color HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definir un rango de valores HSV para el color rojo
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para el rango de color rojo
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # Definir un segundo rango de valores HSV para el color rojo
    lower_red = np.array([160, 100, 100])
    upper_red = np.array([179, 255, 255])

    # Crear una segunda máscara para el rango de color rojo
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    # Combinar las máscaras para detectar todos los tonos de rojo
    mask = mask1 + mask2

    # Aplicar operaciones morfológicas para eliminar el ruido
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((20, 20), np.uint8))

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Dibujar rectángulos alrededor de los objetos rojos detectados y calcular el espacio de color promedio
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filtrar contornos pequeños
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

            # Calcular el espacio de color promedio dentro del contorno en RGB
            roi = frame[y:y + h, x:x + w]
            average_color = np.mean(roi, axis=(0, 1))  # Calcular el promedio en el espacio BGR
            average_color_rgb = average_color[::-1]  # Cambiar el orden de los canales para obtener RGB

            # Mostrar el espacio de color promedio como etiqueta
            label = f'Color Promedio: RGB({int(average_color_rgb[0])}, {int(average_color_rgb[1])}, {int(average_color_rgb[2])})'
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return frame

### 3. Inicializar la cámara web
Esto abre la cámara web. El argumento `0` se utiliza para indicar que se utilizará la primera cámara disponible. Puedes cambiar este valor si tienes múltiples cámaras.

In [ ]:
cap = cv2.VideoCapture(0)

### 4. Bucle principal de captura y procesamiento de imágenes en tiempo real


- El programa entra en un bucle infinito (`while True`) donde captura continuamente frames de la cámara web.
- La variable `ret` indica si se pudo capturar correctamente un frame.
- La función `detect_red_objects(frame)` se llama con cada frame para detectar objetos de color rojo en el frame actual.
- El frame modificado con la detección de objetos se muestra en una ventana llamada "Red Object Detection" utilizando `cv2.imshow`.
- El bucle se ejecuta hasta que se presiona la tecla 'Esc' (código ASCII 27), momento en el que el bucle se rompe.



In [ ]:
while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Llamar a la función para detectar objetos rojos
    frame_with_red_objects = detect_red_objects(frame)

    # Mostrar el frame con los objetos rojos detectados
    cv2.imshow('Red Object Detection', frame_with_red_objects)

    if cv2.waitKey(1) & 0xFF == 27:  # Presionar la tecla 'Esc' para salir
        break

### 5. liberar la cámara y cerrar las ventanas
Cuando se sale del bucle, se libera la cámara web utilizando `cap.release()` y se cierran todas las ventanas de visualización utilizando `cv2.destroyAllWindows()`.

In [ ]:
cap.release()
cv2.destroyAllWindows()

### 6. Código final
Se presenta el código con los comentarios listo para su compilación.

In [6]:
# Importar las bibliotecas
import cv2
import numpy as np

# Función para detectar objetos de color rojo y calcular el espacio de color promedio en RGB
def detect_red_objects(frame):
    # Convertir el frame a espacio de color HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definir un rango de valores HSV para el color rojo
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para el rango de color rojo
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # Definir un segundo rango de valores HSV para el color rojo
    lower_red = np.array([160, 100, 100])
    upper_red = np.array([179, 255, 255])

    # Crear una segunda máscara para el rango de color rojo
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    # Combinar las máscaras para detectar todos los tonos de rojo
    mask = mask1 + mask2

    # Aplicar operaciones morfológicas para eliminar el ruido
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((20, 20), np.uint8))

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Dibujar rectángulos alrededor de los objetos rojos detectados y calcular el espacio de color promedio
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filtrar contornos pequeños
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

            # Calcular el espacio de color promedio dentro del contorno en RGB
            roi = frame[y:y + h, x:x + w]
            average_color = np.mean(roi, axis=(0, 1))  # Calcular el promedio en el espacio BGR
            average_color_rgb = average_color[::-1]  # Cambiar el orden de los canales para obtener RGB

            # Mostrar el espacio de color promedio como etiqueta
            label = f'Color Promedio: RGB({int(average_color_rgb[0])}, {int(average_color_rgb[1])}, {int(average_color_rgb[2])})'
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return frame

# Inicializar la cámara web
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Llamar a la función para detectar objetos rojos
    frame_with_red_objects = detect_red_objects(frame)

    # Mostrar el frame con los objetos rojos detectados
    cv2.imshow('Red Object Detection', frame_with_red_objects)

    if cv2.waitKey(1) & 0xFF == 27:  # Presionar la tecla 'Esc' para salir
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

